In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta

In [56]:
datastocks = pd.read_csv('data_snp_stocks.csv')
datastocks['date'] = pd.to_datetime(datastocks['date'], format = 'mixed')
datastocks.set_index('date', inplace=True)
tickers = datastocks['Name'].unique()

In [69]:
snp500 = pd.read_csv('spx.csv',)
#snp500['Date'] = pd.to_datetime(snp500['Date'], format='mixed')

In [70]:
snp500

,date,close
0,02-Jan-86,209.59
1,03-Jan-86,210.88
2,06-Jan-86,210.65
3,07-Jan-86,213.80
4,08-Jan-86,207.97
...,...,...
8187,25-Jun-18,2717.07
8188,26-Jun-18,2723.06
8189,27-Jun-18,2699.63
8190,28-Jun-18,2716.31


In [71]:
if 'MarketReturn' not in snp500.columns:
    snp500['MarketReturn'] = np.log(snp500['close'] / snp500['close'].shift(1))

In [54]:
data = datastocks.merge(snp500, left_on='date', right_on='Date')

In [55]:
data

,open,high,low,close,volume,Name,LogReturn,Date,Price,MarketReturn
0,43.04,44.55,43.03,44.550,40644263,AAL,0.034018,2014-06-20,1.963,0.000000
1,44.30,44.88,43.80,44.360,18457895,AAL,-0.004274,2014-06-23,1.963,0.006645
2,44.14,44.61,42.76,43.179,13252583,AAL,-0.026984,2014-06-24,1.950,-0.005115
3,42.90,44.00,42.30,43.910,10944639,AAL,0.016788,2014-06-25,1.960,0.001532
4,43.96,44.60,43.50,44.550,11175082,AAL,0.014470,2014-06-26,1.957,-0.002042
...,...,...,...,...,...,...,...,...,...,...
453966,76.84,78.27,76.69,77.820,2982259,ZTS,0.014106,2018-02-01,2.822,0.021491
453967,77.53,78.12,76.73,76.780,2595187,ZTS,-0.013454,2018-02-02,2.762,0.041773
453968,76.64,76.92,73.18,73.830,2962031,ZTS,-0.039179,2018-02-05,2.649,-0.017216
453969,72.74,74.56,72.13,73.270,4924323,ZTS,-0.007614,2018-02-06,2.695,0.004835


In [47]:
datastocks.dtypes

open      float64
high      float64
low       float64
close     float64
volume      int64
Name       object
dtype: object

In [48]:
datastocks.head()

,open,high,low,close,volume,Name
date,,,,,,
2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [49]:
datastocks.isna().sum()

open      11
high       8
low        8
close      0
volume     0
Name       0
dtype: int64

In [50]:
#data.sort_values(by=['Name', 'date'], inplace=True)
datastocks['LogReturn'] = datastocks.groupby('Name')['close'].transform(lambda x: np.log(x / x.shift(1)))

In [51]:
datastocks['LogReturn'].isna().sum()

np.int64(505)

In [52]:
datastocks = datastocks[datastocks['LogReturn'].notna()]

To perform the rolling regression we would need to solve the following equation:

$r_i -r_f = \alpha_i + \beta_i (R_M - r_f) + e_i$

Where:

$r_i$ is the return of the stock $i$ \
$r_f$ is the return of the risk free asset \
$\alpha_i$ is the intercept of the regression, or the excess return \
$\beta_i$ is the regression coefficient \
$R_M$ is the return of the market (log-return) \
$e_i$ is the specific risk of stock $i$


In [37]:
df_coeff = pd.DataFrame(columns = ['Ticker', 'alpha', 'beta', 'r_squared', 'specific_risk'])

X = snp500.iloc[:,1].values

for ticker in tickers:
    data = data180[data180['Name'] == ticker]
    #data = data.merge(snp500, left_index=True, right_on='Date')
    #data['LogReturn_snp'] = np.log(data['Price'] / data['Price'].shift(1))
    #data = data.dropna()
    y = data['LogReturn']
    X = sm.add_constant(X)
    model = sm.OLS(X, y)
    result = model.fit()
    df_coeff.append({'Ticker': ticker, 'alpha': result.params[0], 'beta': result.params[1], 'r_squared': result.rsquared, 'specific_risk': result.mse_resid.std(), 'systematic_risk': result.params[1] ** 2 * X.std() ** 2}, ignore_index=True)


ValueError: You are trying to merge on int64 and datetime64[ns] columns for key 'Date'. If you wish to proceed you should use pd.concat